## Predicting Energy Consumption from PJM (in MW)

Will also be comparing energy consumption from the East and West Regions of the United States as well as their projected energy consumption.

In [26]:
import pandas as pd
import seaborn as sbn
import plotly as plt
import numericalmethods as nm

In [ ]:
# Importing the csv files and assigning them as dataframes
dfw = pd.read_csv('PJMW_hourly.csv')
dfe = pd.read_csv('PJME_hourly.csv')

West-region

In [ ]:
# Using numerical methods to create a model from each dataset


East-region

In [ ]:
# Using numerical methods to create a model from each dataset
